# One-time pad adventures

## Encryption and Decryption

Let's start by creating some fake "truly" random bytes.

In [ ]:
from Crypto.Random import get_random_bytes

padBytes = get_random_bytes(100)

These bytes are pre-computed ahead of time and shared between the people sending messages between each other. They must remain secret forever.

Now let's define a message

In [ ]:
plaintext = b'Kill the king at midnight'

Let's define the encryption / decryption function.

In [295]:
def encrypt_decrypt(payload, key):
    output = bytearray()
    for i in range(0, len(payload)):
        outputByte = payload[i] ^ key[i]
        output.append(outputByte)
    return output

And test:

In [ ]:
cipherText = encrypt_decrypt(plaintext, padBytes )
cipherText

And check that we can recover the plain-text:

In [ ]:
encrypt_decrypt(cipherText, padBytes)

## Attack: Using the same key twice

Using the same key twice completely destroys security. Suppose I have two messages $m_0$ and $m_1$. They are the same length and encrypted with the same key, $k$.

Then by commutativity of XOR and the property that $\forall A: A \oplus A = 0$:

$$(m_0 \oplus  k) \oplus  (m_1 \oplus  k) \\ = (m_0 \oplus  m_1) \oplus  (k \oplus  k) \\ = (m_0 \oplus  m_1)  $$

The key cancels out and we're left with the logical XOR of the two plain-texts. If we know one of them, we can trivially compute the other.

Even if you don't know either of them, you can recover both texts by looking at where the zeros are. Everywhere you have a matching letter in the same position, they 
will cancel each other out and we'll end up with a zero. 

"e" is the most common English letter. So "e" is the mostly likely to overlap between messages. Solve by filling in the letters like a Sudoku puzzle. 

Even though this full plain-text recovery attack exists, let's show there's a problem using our semantic security framework!

In [ ]:
class OTP_ReusedKey:
    def __init__(self):
        self.key = get_random_bytes(100)

    def encrypt(self, data):
        return encrypt_decrypt(data, self.key)         


In the above snippet, we fix the key at the start of the game and just repeatedly encrypt with the same key.

The strategy below is the same as what we saw with RC4. On the first trial, send the same message twice to get the encryption.

On all other attempts, send the pair. The cipher texts will always ben the same so we can just check the cipherTexts match $m_0$ or not.

In [ ]:
class OTP_ReusedKey_Strategy:
    def getMessages(self, trialNumber):
        m0 = b'Kill the king at 2am'
        m1 = b'Kill the king at 3am'

        if trialNumber == 0:
            return m0, m0
        else:
            return m0, m1 

    def challenge(self, challenge, trialNumber):
        if trialNumber == 0:
            self.m0Encrypted = challenge
            return 0
        else:
            if challenge == self.m0Encrypted:
                return 0
            else:
                return 1

Let's check our logic is right!

In [ ]:
from  SemanticSecurityGame import SemanticSecurityGame
game = SemanticSecurityGame(OTP_ReusedKey(), OTP_ReusedKey_Strategy())
game.runGame(200)

## Attack: A bias random number generator

Suppose that we don't produce completely random bytes, can we break the OTP?

This attack is quite fiddly, but stay with me. First we need to create a bias source of random bits.

In [294]:
class biasRNG:
    def __init__(self, bias):
        self.bias = bias

    def Clock(self):
       byte = self.generateBiasBit() 
       for i in range(0, 7):
           byte = byte * 2
           byte = byte + self.generateBiasBit()
       
       return byte 
    
    def get_many(self, count):
        result = bytearray()
        for i in range(0, count):
            result.append(self.Clock())
        return result
    
    def generateBiasBit(self):
        import random 
        return random.choice(self.bias)

Okay, let's look at our bias bit generator in action. We pass in an array of zeros and ones to define the bias of bits.

In [259]:
bias = biasRNG([0, 1])

bias.Clock()


216

And all ones:

In [260]:
bias = biasRNG([1])

bias.Clock()

255

And all zeros:

In [262]:
bias = biasRNG([0])

bias.Clock()

0

And now for something more subtle:

In [269]:
biasTable = ([0] * 50) + ([1] * 51)
bias = biasRNG(biasTable)

bias.Clock()

0

### Making the attack

So how will our attack work?

We'll pass in $m_0$ as a long string of zeros and $m_1$ as a long string of ones. 

To determine $m_0$ from $m_1$ we'll simply count the number of bits in the returned cipher-text. If 50% of the bits or more are 1s, we say it's $m_0$.

First we set-up the rules of the game:

In [290]:
class OTP_BiasKey:
    def __init__(self, biasTable):
        self.rng =  biasRNG(biasTable)

    def encrypt(self, data):
        key = self.rng.get_many(len(data))

        return encrypt_decrypt(data, key)    

Now the attack code:

In [288]:
class OTP_BiasKey_Strategy:
    def getMessages(self, trialNumber):
        m0 = [0] * 1000
        m1 = [255] * 1000

        return m0, m1 

    def challenge(self, challenge, trialNumber):
       accumulator = 0
       for byte in challenge:
           accumulator = accumulator + byte.bit_count()
        
       if accumulator > (500*8):
            return 0
       else:
            return 1
       


{'Trials': 200, 'Wins': 99}

And now we make our attack!

In [293]:
game = SemanticSecurityGame(OTP_BiasKey([1]), OTP_BiasKey_Strategy())
game.runGame(200)

{'Trials': 200, 'Wins': 101}